<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/Experiments/CosineSimilarity2Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tensorflow-project/FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 1108, done.
remote: Counting objects: 100% (379/379), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 1108 (delta 296), reused 220 (delta 180), pack-reused 729
Receiving objects: 100% (1108/1108), 143.32 MiB | 16.95 MiB/s, done.
Resolving deltas: 100% (802/802), done.
Updating files: 100% (30/30), done.


In [ ]:
import sys
import os
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('ps')

py_file_location = "/content/FineTuning/two_concepts"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

import textual_inversion_two_concepts as txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
13911/13911 [==============================] - 0s 1us/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


136824240/136824240 [==============================] - 1s 0us/step


In [ ]:
### Mount Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
### define paths where to store the arrays for cosine similarities
path1 = '/content/drive/MyDrive/cosine_broc.npy'
path2 = '/content/drive/MyDrive/cosine_emoji.npy'
path3 = '/content/drive/MyDrive/cosine_combi.npy'

In [ ]:
### embedding arrays for cosine similarity
broccoli_embedding = []
emoji_embedding = []
combined_embedding = []
cosine_similarity_broc = []
cosine_similarity_emoji = []
cosine_similarity_combi = []
cosine_broc = []
cosine_emoji = []
cosine_combi = []
### x axis af plot
epoch_numb = [0]


In [ ]:
def update():
  """This function updates three sets of cosine similarity values and saves them in three separate numpy files.
  The first set of cosine similarity values is calculated between the "broccoli" embedding and the placeholder token for "broccoli". The resulting array is saved in a numpy file specified by the variable path1.
  The second set of cosine similarity values is calculated between the "emoji" embedding and the placeholder token for "emoji". The resulting array is saved in a numpy file specified by the variable path2.
  The third set of cosine similarity values is calculated between the combination of the "broccoli" embedding and a modified "emoji" embedding, and the placeholder token for the combined embeddings. The resulting array is saved in a numpy file specified by the variable path3.

  Args:
  - None

  Returns:
  - None
  """
  cosine_similarity_broc.append(txt.cosine_sim(txt.get_embedding("broccoli"), txt.get_embedding(txt.placeholder_token_broccoli)))
  cosine_broc = np.array(cosine_similarity_broc)
  np.save(path1, cosine_broc)


  cosine_similarity_emoji.append(txt.cosine_sim(txt.get_embedding("emoji"), txt.get_embedding(txt.placeholder_token_emoji)))
  cosine_emoji = np.array(cosine_similarity_emoji)
  np.save(path2, cosine_emoji)

  combi = txt.get_embedding("broccoli") + (0.3*(txt.get_embedding("emoji") - txt.get_embedding("broccoli")))
  
  cosine_similarity_combi.append(txt.cosine_sim(combi, txt.get_embedding(txt.placeholder_token_combined)))
  cosine_combi = np.array(cosine_similarity_combi)
  np.save(path3, cosine_combi)



In [ ]:
#save the similarities in arrays before training
update()

In [ ]:
### training
for i in range (3):
  print( f"Epoch:{i+1}" )
  txt.trainer.fit(
      txt.train_ds,
      epochs=1,
  )

  update()
  ###expand x axis
  epoch_numb.append(i+1)

Epoch:1
220/220 [==============================] - 442s 2s/step - loss: 0.0627
Epoch:2
220/220 [==============================] - 346s 2s/step - loss: 0.0640
Epoch:3
220/220 [==============================] - 346s 2s/step - loss: 0.0569


In [ ]:
### load the arrys from drive
cosine_broc = np.load(path1, allow_pickle=True)
cosine_emoji = np.load(path2, allow_pickle=True)
cosine_combi = np.load(path3, allow_pickle=True)

In [ ]:
print(epoch_numb)
print(cosine_broc)

[0, 1, 2, 3]
[0.99999994 0.9836218  0.962476   0.94018704]


In [ ]:
def plot(x_values, y_values, color, xlabel, ylabel, title):
    """Plot a line graph using Matplotlib.

    Args:
    - x_values (array-like): x-coordinates of data points to be plotted
    - y_values (array-like): y-coordinates of data points to be plotted
    - color (str): color of the line
    - xlabel (str): label of the x-axis
    - ylabel (str): label of the y-axis
    - title (str): title of the plot

    Returns:
    - None

    Raises:
    - None
    """
    plt.figure()

    plt.figure()  # Create a new figure
    plt.plot(x_values, y_values, color)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()
     


In [ ]:
# Extract x and y values from the data
x_values = epoch_numb
y_values = cosine_broc

# Plot data using Matplotlib
plot(x_values = x_values, y_values = y_values, color='darkblue', xlabel = 'Epochs', ylabel = 'Cosine Similarity', title = 'Cosine Similarity between basis and placeholder token broccoli')
plt.show()

# Save the plot as a file
plt.savefig('/content/drive/MyDrive/EPS/CosineBrocc.eps', format='eps')
plt.savefig('/content/drive/MyDrive/EPS/CosineBrocc.pdf', format='pdf')

In [ ]:
x_values = epoch_numb
y_values = cosine_emoji

# Plot data using Matplotlib
plot(x_values = x_values, y_values = y_values, color='darkblue', xlabel = 'Epochs', ylabel = 'Cosine Similarity', title = 'Cosine Similarity between basis and placeholder token emoji')

# Save the plot as a file
plt.savefig('/content/drive/MyDrive/EPS/CosineEmoji.eps', format='eps')
plt.savefig('/content/drive/MyDrive/EPS/CosineEmoji.pdf', format='pdf')
plt.show()

In [ ]:
x_values = epoch_numb
y_values = cosine_combi

# Plot data using Matplotlib
plot(x_values = x_values, y_values = y_values, color='darkblue', xlabel = 'Epochs', ylabel = 'Cosine Similarity', title = 'Cosine Similarity between basis and placeholder token combined')

# Save the plot as a file
plt.savefig('/content/drive/MyDrive/EPS/CosineCombi.eps', format='eps')
plt.savefig('/content/drive/MyDrive/EPS/CosineCombi.pdf', format='pdf')